In [2]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from src.processors.Preprocessor import Preprocessor
from src.model_experiemtation.ModelExperimentor import TFModelExperimentor

import datetime
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from sklearn.model_selection import train_test_split
import tensorflow as tf
from time import sleep

import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, LSTM
from tensorflow.keras import Sequential

from tensorflow.keras.losses import Huber, MeanSquaredError

#validate GPU detection
print("TensorFlow version:", tf.__version__)
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)


2023-05-25 15:56:15.003364: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 15:56:15.058711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 15:56:17.932586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [4]:
PATH = '/home/mds8301/gaby_test/processors/unit_test_processor.pkl'
proc = Preprocessor().load_processor(PATH)
proc.one_hot_encode(labels = ['event', 'sensor'])

## initial prototyping plan
* query trials below 20 for training
* query trials above 20 for testing and prediction
* drop mouse_id 




In [5]:
data_under_20 = proc.processed_data.query('trial<=20')
data_over_20 = proc.processed_data.query('trial>20')


In [6]:
cols_to_drop = ['signal', 'mouse_id']
X_train, y_train = data_under_20.drop(columns = cols_to_drop), data_under_20.signal
X_test, y_test = data_over_20.drop(columns = cols_to_drop), data_over_20.signal

In [7]:
X_train

,day,time,trial,event_avoid,event_cue,event_escape,event_shock,sensor_D1,sensor_D2,sensor_DA
0,7,-25.000000,0,0,0,0,1,0,1,0
1,7,-24.999018,0,0,0,0,1,0,1,0
2,7,-24.998034,0,0,0,0,1,0,1,0
3,7,-24.997051,0,0,0,0,1,0,1,0
4,7,-24.996067,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
228880,7,19.996067,4,0,0,0,1,0,0,1
228881,7,19.997051,4,0,0,0,1,0,0,1
228882,7,19.998034,4,0,0,0,1,0,0,1
228883,7,19.999018,4,0,0,0,1,0,0,1


In [8]:


sequential_model = Sequential(
    [
        Dense(128, activation='relu', name="dense_1"),
        Dense(1, activation='relu', name="dense_2")
    ]
)


2023-05-25 15:57:41.859506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38222 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:06:00.0, compute capability: 8.0
2023-05-25 15:57:41.861232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38222 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


In [10]:
model_id = 'sequential_prototype'
date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
training_log_dir = "/projects/p31961/dopamine_modeling/results/logs/training_logs/" 
logs_dir = f"{training_log_dir}/{model_id}/{date_time}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [11]:
sequential_model.compile(
    optimizer="adam", loss=Huber())
sequential_model.fit(X_train, y_train, epochs=100,
                 batch_size=30, callbacks=[tensorboard_callback])
prediction = sequential_model.predict(X_test)
sequential_model.evaluate(X_test, y_test)

Epoch 1/100


2023-05-25 15:58:33.524330: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-25 15:58:34.049647: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2af2dc737600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-25 15:58:34.049709: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2023-05-25 15:58:34.049727: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2023-05-25 15:58:34.061275: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-25 15:58:34.316372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-25 15:5

 192752/2203400 [=>............................] - ETA: 1:02:47 - loss: nan